In [1]:
'''
Streamlined Randomized Benchmarking (SRB)
to OpenQASM & Qiksit
1 qubit  sequence 
'''
import trueq as tq
import numpy as np
from pprint import pprint
import trueq.simulation as tqs
print('True-Q ver',tq.__version__)

True-Q ver 2.10.0


In [19]:
gateF='trueq_gates_zxzxz.yaml'
print('\nM: transpile to QubiC basis gates and use chip mapping:',gateF)
config=tq.Config.from_yaml(gateF)
print('trueq_gatecfg',config)

qid=1 # qubit id must match gates-.yaml
circCol = tq.make_srb(qid, [1],n_circuits=1, twirl="U")
circ=circCol[0]


M: transpile to QubiC basis gates and use chip mapping: trueq_gates_zxzxz.yaml
trueq_gatecfg Mode: ZXZXZ
Dimension: 2
Gates:
- Z(phi):
    Hamiltonian:
    - [Z, phi]
    Involving: {'(0,)': (), '(1,)': ()}
- X(phi):
    Hamiltonian:
    - [X, phi]
    Involving: {'(5,)': (), '(1,)': ()}
- CNOT:
    Matrix:
    - [1.0, 0.0, 0.0, 0.0]
    - [0.0, 1.0, 0.0, 0.0]
    - [0.0, 0.0, 0.0, 1.0]
    - [0.0, 0.0, 1.0, 0.0]
    Involving: {'(0, 1)': ()}



In [20]:
circ  # will display interactive circuit

Circuit(Cycle((1,): Gate(X, Y, ...)),Cycle(marker=1),Cycle((1,): Gate(X, Y, ...)),Cycle((1,): Meas(), marker=2))

In [22]:
circ.draw()

DisplayWrapper(<svg xmlns="http://w...)

In [23]:
transpiler = tq.Compiler.from_config(config)
transCol = transpiler.compile(circCol)
print('M: transpile done')

circ_compiled = transCol[0]
circ_compiled # will display interactive gates

M: transpile done


Circuit(Cycle((1,): Z(phi)),Cycle((1,): X(phi)),Cycle((1,): Z(phi)),Cycle((1,): X(phi)),Cycle((1,): Z(phi)),Cycle(marker=1),Cycle((1,): Z(phi)),Cycle((1,): X(phi)),Cycle((1,): Z(phi)),Cycle((1,): X(phi)),Cycle((1,): Z(phi)),Cycle((1,): Meas(), marker=2))

In [24]:
print('VZ angles directly from TQ transpiler ')
for i,cycle in enumerate(circ_compiled):
    gates=cycle.gates
    if len(gates)<1: continue
    for qidg in gates.keys(): # 1 or 2 qubit tuple
        gate=cycle.gates[qidg]
        name=gate.name.upper()
        if name=='X': continue
        phi_deg=gate.parameters['phi']
        phi=phi_deg/57.30 % np.pi*2
        print('  qidg',qidg,'name=',name, 'phi/rad=%.3f  =%.0d deg'%(phi,phi_deg))


VZ angles directly from TQ transpiler 
  qidg (1,) name= Z phi/rad=0.730  =-159 deg
  qidg (1,) name= Z phi/rad=4.568  =130 deg
  qidg (1,) name= Z phi/rad=4.239  =-238 deg
  qidg (1,) name= Z phi/rad=2.045  =-121 deg
  qidg (1,) name= Z phi/rad=4.568  =130 deg
  qidg (1,) name= Z phi/rad=5.554  =-20 deg


## continue transformations in Qiskit

In [28]:
circ_qiskit=circ.to_qiskit()
print('same circuit exported to Qiskit')
print(circ_qiskit)

same circuit exported to Qiskit
                                                                         »
q_0: ────────────────────────────────────────────────────────────────────»
     ┌───────────┐┌───────────────────┐┌───────────┐┌───────────────────┐»
q_1: ┤ P(-2.777) ├┤ R(π/2,7.2394e-10) ├┤ P(2.2842) ├┤ R(π/2,7.2394e-10) ├»
     └───────────┘└───────────────────┘└───────────┘└───────────────────┘»
c: 1/════════════════════════════════════════════════════════════════════»
                                                                         »
«                   ░                                                 »
«q_0: ──────────────░─────────────────────────────────────────────────»
«     ┌───────────┐ ░ ┌────────────┐┌───────────────────┐┌───────────┐»
«q_1: ┤ P(2.1193) ├─░─┤ P(-2.1193) ├┤ R(π/2,7.2394e-10) ├┤ P(2.2842) ├»
«     └───────────┘ ░ └────────────┘└───────────────────┘└───────────┘»
«c: 1/════════════════════════════════════════════════════════════════»
«          

In [40]:
from qiskit.converters import circuit_to_dag, dag_to_circuit

def print_dag_layers(qc):
  qdag = circuit_to_dag(qc)  
  dagLay=qdag.layers()
  print('Show %d layers as lists of gates, format: gateName qubits[] params[]; '%qdag.properties()['depth'])
  phase3z=[]
  for k,lay in enumerate(dagLay):
    for op in lay['graph'].op_nodes():
      qL=[qub._index for qub in  op.qargs]
      parV=[ float('%.6f'%v)  for v in op._op._params]
      #if op.name=='r': continue # skip X90 gates
      print(k,'  ',op.name, 'q'+str(qL), parV)
      if op.name=='p': phase3z.append(parV[0])
  print('end-of-cycles')
  return phase3z

print('\nDecomposition of circuit DAG into cycles')
circ_phase3z=print_dag_layers(circ_qiskit)
print('phase3z=',circ_phase3z)


Decomposition of circuit DAG into cycles
Show 13 layers as lists of gates, format: gateName qubits[] params[]; 
0    p q[1] [-2.776968]
1    r q[1] [1.570796, 0.0]
2    p q[1] [2.284244]
3    r q[1] [1.570796, 0.0]
4    p q[1] [2.119283]
5    barrier q[0, 1] []
6    p q[1] [-2.119283]
7    r q[1] [1.570796, 0.0]
8    p q[1] [2.284244]
9    r q[1] [1.570796, 0.0]
10    p q[1] [-0.364625]
11    barrier q[0, 1] []
12    measure q[1] []
end-of-cycles
phase3z= [-2.776968, 2.284244, 2.119283, -2.119283, 2.284244, -0.364625]


In [7]:
round_trip, meta_data = tq.interface.qiskit.to_trueq_circ(circ.to_qiskit())
round_trip

Circuit(Cycle((5,): PhaseGate(theta)),Cycle((5,): RGate(theta, phi)),Cycle((5,): PhaseGate(theta)),Cycle((5,): RGate(theta, phi)),Cycle((5,): PhaseGate(theta)),Cycle((5,): PhaseGate(theta), marker=1),Cycle((5,): RGate(theta, phi), marker=1),Cycle((5,): PhaseGate(theta), marker=1),Cycle((5,): RGate(theta, phi), marker=1),Cycle((5,): PhaseGate(theta), marker=1),Cycle((5,): Meas(), marker=2))

In [8]:
sim = tq.Simulator()

In [9]:
op_tq = sim.operator(circ_compiled)
op_qiskit = sim.operator(round_trip)

In [10]:
op_tq.mat()

array([[ 5.55111512e-17+1.00000000e+00j, -2.07768551e-16+2.83004851e-16j],
       [ 2.07768551e-16+2.83004851e-16j,  5.55111512e-17-1.00000000e+00j]])

In [11]:
op_qiskit.mat()

array([[ 1.76873928e-08+1.00000000e+00j, -6.65358237e-09+1.88533759e-08j],
       [ 6.65358237e-09+1.88533759e-08j,  1.76873928e-08-1.00000000e+00j]])

In [41]:
assert np.allclose(op_tq.mat(), op_qiskit.mat())

AssertionError: 